# Árvores II - Tarefa 2

### 1. Carregar as bases

Vamos carregar as bases lidas na tarefa passada. Se você salvou essas bases em arquivo texto, basta fazer a leitura com o comando ```pd.read_csv``` das seguintes bases:

- X_train
- Y_train
- X_test
- Y_test

Não se esqueça de considerar a leitura dos índices dos arquivos no ```read_csv()```!

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

**Leitura das bases**

In [2]:
x_treino = pd.read_csv('./x_treino.csv').drop(columns='Unnamed: 0')
x_teste  = pd.read_csv('./x_teste.csv').drop(columns='Unnamed: 0')

y_treino = pd.read_csv('./y_treino.csv').drop(columns='Unnamed: 0')
y_teste  = pd.read_csv('./y_teste.csv').drop(columns='Unnamed: 0')

x_treino.set_index([x_treino.index, 'subject'], inplace=True)
x_teste.set_index([x_teste.index, 'subject'], inplace=True)

### 2. Divisão da base em Treino, Validação e Teste

A base já se encontra dividida em Treino e Validação. O que vamos fazer então é extrair uma base de Validação da base de Treino.

Extraia 25% da base de treino como base de validação.

In [3]:
x_treino, x_validacao, y_treino, y_validacao = train_test_split(x_treino, y_treino, test_size=0.25)

### 3. Melhores 3 variáveis

Rode uma árvore com profundidade máxima igual a 4 para prever a atividade humana com todas as variáveis.
Observe a importância das variáveis e considere as 3 variáveis com maior importância para os próximos passos.
Dica: utilize o atributo ```clf.feature_importances_``` da árvore treinada.

In [4]:
clf = DecisionTreeClassifier(max_depth=4).fit(x_treino, y_treino)

importancias = clf.feature_importances_

importancias

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.01084601, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.2865918 , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.12183042,
       0.        , 0.        , 0.        , 0.        , 0.     

In [5]:
# Crio um dataframe pra saber qual variável corresponde a cada valor
importancias_variaveis = pd.DataFrame({
    'nome_coluna': x_treino.columns,
    'importance': importancias
})

# Pego o nome das 3 colunas mais significativas para o modelo
colunas_significativas = importancias_variaveis.sort_values(by = 'importance').tail(3)['nome_coluna']

colunas_significativas

559               angle(Y,gravityMean)
389    fBodyAccJerk-bandsEnergy()-1,16
52                 tGravityAcc-min()-X
Name: nome_coluna, dtype: object

### 4. Construa uma árvore com as 3 melhores variáveis

Utilizando as três variáveis encontradas acima, construa uma árvore de decisão. Encontre o melhor ```ccp_alpha``` utilizando a base de validação, conforme a estrutura que vimos em aula.

In [6]:
# Reduzo os dataframes para as 3 colunas selecionadas no passo anterior
x_treino    = x_treino[colunas_significativas]
x_validacao = x_validacao[colunas_significativas]
x_teste     = x_teste[colunas_significativas]

# Crio o modelo
modelo = DecisionTreeClassifier()

# Pego a lista de ccps_alpha
ccps = modelo.cost_complexity_pruning_path(x_treino, y_treino).ccp_alphas

**Cálculo do melhor ccp na base de validação**

In [7]:
%%time

lista_modelo = []
lista_scores = []


for ccp in ccps:
    
    # Treino do modelo (com base de treino)
    modelo = DecisionTreeClassifier(ccp_alpha=ccp).fit(x_treino, y_treino)
    lista_modelo.append(modelo)
    
    # Cálculo da acurácia (na base de validação)
    lista_scores.append( modelo.score(x_validacao, y_validacao) )

    
# Crio um dataframe para avaliar os resultados
df_resultados = pd.DataFrame({
    
    'modelo': lista_modelo,
    'ccp': ccps,
    'score': lista_scores
    
})

df_resultados

CPU times: total: 6.86 s
Wall time: 7.59 s


,modelo,ccp,score
0,DecisionTreeClassifier(),0.000000,0.816649
1,DecisionTreeClassifier(ccp_alpha=0.00010749004...,0.000107,0.811752
2,DecisionTreeClassifier(ccp_alpha=0.00011872763...,0.000119,0.815560
3,DecisionTreeClassifier(ccp_alpha=0.00011917374...,0.000119,0.819369
4,DecisionTreeClassifier(ccp_alpha=0.00012092629...,0.000121,0.813384
...,...,...,...
387,DecisionTreeClassifier(ccp_alpha=0.04234435873...,0.042344,0.727421
388,DecisionTreeClassifier(ccp_alpha=0.05133468026...,0.051335,0.644178
389,DecisionTreeClassifier(ccp_alpha=0.10237862178...,0.102379,0.523395
390,DecisionTreeClassifier(ccp_alpha=0.15980638525...,0.159806,0.367247


**Avaliação dos resultados**

Avalio os modelos que ficaram com a maior acurácia

In [8]:
df_resultados.sort_values(by = 'score', ascending=False)

,modelo,ccp,score
214,DecisionTreeClassifier(ccp_alpha=0.00026371234...,0.000264,0.849837
223,DecisionTreeClassifier(ccp_alpha=0.00027208416...,0.000272,0.849293
225,DecisionTreeClassifier(ccp_alpha=0.00027208416...,0.000272,0.849293
231,DecisionTreeClassifier(ccp_alpha=0.00027208416...,0.000272,0.848749
216,DecisionTreeClassifier(ccp_alpha=0.00026510764...,0.000265,0.848749
...,...,...,...
387,DecisionTreeClassifier(ccp_alpha=0.04234435873...,0.042344,0.727421
388,DecisionTreeClassifier(ccp_alpha=0.05133468026...,0.051335,0.644178
389,DecisionTreeClassifier(ccp_alpha=0.10237862178...,0.102379,0.523395
390,DecisionTreeClassifier(ccp_alpha=0.15980638525...,0.159806,0.367247


Pego o modelo que tem a maior acurácia

In [9]:
melhor_modelo = df_resultados.loc[ df_resultados['score'].idxmax(), 'modelo']

### 5. Avaliação do modelo

Avalie a árvore encontrada no item anterior na base de testes.

Foi possível obter uma acurácia de 65% nos dados de teste

In [10]:
# Testo a acurácia do modelo em todas as bases

print(f'Acurácia treino    = {melhor_modelo.score(x_treino, y_treino)}')
print(f'Acurácia validação = {melhor_modelo.score(x_validacao, y_validacao)}')
print(f'Acurácia teste     = {melhor_modelo.score(x_teste, y_teste)}')

Acurácia treino    = 0.9406856520950481
Acurácia validação = 0.8498367791077258
Acurácia teste     = 0.6541072640868975
